<a href="https://colab.research.google.com/github/Vaibhavi-Lakhani/COMP-8700-Assignment-II/blob/main/Que1_Task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#pydot is an interface to Graphviz and Graphviz is for drawing graph and its visualization
!apt-get -qq install -y graphviz && pip install pydot
import pydot as pydot
import numpy as np

In [ ]:
#Node Class
import pydot as pydot
class Node:
    goal_state = [0, 0, 0]
    num_of_instances = 0

    def __init__(self, state, parent, action, depth):
        self.state = state
        self.parent = parent
        self.action = action
        self.depth = depth
        if self.is_killed():
            color = "red"
        elif self.goal_test():
            color = "green"
        else:
            color = "orange"
        self.graph_node = pydot.Node(str(self), style="filled", fillcolor=color)
        Node.num_of_instances += 1

    def __str__(self):
        return str(self.state)

    def goal_test(self):
        if self.state == self.goal_state:
            return True
        return False

    def is_valid(self):
        missionaries = self.state[0]
        cannibals = self.state[1]
        boat = self.state[2]
        if missionaries < 0 or missionaries > 3:
            return False
        if cannibals < 0 or cannibals > 3:
            return False
        if boat > 1 or boat < 0:
            return False
        return True

    def is_killed(self):
        missionaries = self.state[0]
        cannibals = self.state[1]
        if cannibals > missionaries > 0:
            return True
        # Check for the other side
        if cannibals < missionaries < 3:
            return True

    def generate_child(self):
        children = []
        depth = self.depth + 1
        op = -1  # Subtract
        if self.state[2] == 0:
            op = 1  # Add
        for x in range(3):
            for y in range(3):
                # by_move = "Move %s missionaries and %s cannibals %s" % (x, y, boat_move)
                new_state = self.state.copy()
                new_state[0], new_state[1], new_state[2] = new_state[0] + op * x, new_state[1] + op * y, new_state[
                    2] + op * 1
                action = [x, y, op]
                new_node = Node(new_state, self, action, depth)
                if 1 <= x + y <= 2:
                    children.append(new_node)
        return children

    def find_solution(self):
        solution = []
        solution.append(self.action)
        path = self
        while path.parent != None:
            path = path.parent
            solution.append(path.action)
        #solution = solution[:-1]
        solution.reverse()
        return solution

In [ ]:
#DrawLegend
def draw_legend(graph):
    graphlegend = pydot.Cluster(graph_name="legend", label="Labels", fontsize="20", color="red",
                                fontcolor="blue", style="filled", fillcolor="white", fontname='Arsenal')

    legend1 = pydot.Node('Processed State', shape="plaintext", fontname='Arsenal', fontsize="20")
    graphlegend.add_node(legend1)
    legend2 = pydot.Node("Killed State ", shape="plaintext", fontname='Arsenal', fontsize="20")
    graphlegend.add_node(legend2)
    legend3 = pydot.Node('Impossible State', shape="plaintext", fontname='Arsenal', fontsize="20")
    graphlegend.add_node(legend3)
    legend4 = pydot.Node('Goal State', shape="plaintext", fontname='Arsenal', fontsize="20")
    graphlegend.add_node(legend4)

    node1 = pydot.Node("1", style="filled", fillcolor="orange", label="")
    graphlegend.add_node(node1)
    node2 = pydot.Node("2", style="filled", fillcolor="red", label="")
    graphlegend.add_node(node2)
    node3 = pydot.Node("3", style="filled", fillcolor="blue", label="")
    graphlegend.add_node(node3)
    node4 = pydot.Node("4", style="filled", fillcolor="green", label="")
    graphlegend.add_node(node4)

    graph.add_subgraph(graphlegend)
    graph.add_edge(pydot.Edge(legend1, legend2, style="invis"))
    graph.add_edge(pydot.Edge(legend2, legend3, style="invis"))
    graph.add_edge(pydot.Edge(legend3, legend4, style="invis"))

    graph.add_edge(pydot.Edge(node1, node2, style="invis"))
    graph.add_edge(pydot.Edge(node2, node3, style="invis"))
    graph.add_edge(pydot.Edge(node3, node4, style="invis"))

In [ ]:

###Breadth-First Search Algorithm

def bfs(initial_state, mode="BFS"):
  graph = pydot.Dot(graph_type='digraph', label="Missionaries and Cannibals (BFS)", fontsize="30", color="red", fontcolor="black", fontname='Arsenal', fillcolor="black")
  start_node = Node(initial_state, None, None, 0) #depth is zero                                                                                         
  if start_node.goal_test():
    return start_node.find_solution()
  frontier = [start_node]
  explored = []
  killed = []

  print("The Starting node is \nDepth = %d" % start_node.depth)
  print(str(start_node.state))

  while frontier:
      node = frontier.pop(0)
      print("\nThe node selected to expand is\nDepth = " + str(node.depth) + "\n" + str(node.state) + "\n")

      explored.append(node.state)
      graph.add_node(node.graph_node)

      if node.parent:
          diff = np.subtract(node.parent.state, node.state)
          if node.parent.state[2] == 0:
              diff[0], diff[1] = -diff[0], -diff[1]
          graph.add_edge(
              pydot.Edge(node.parent.graph_node, node.graph_node, label=str(diff[0]) + "M  " + str(diff[1]) + "C",
                          fontsize='10', fontcolor='#cc0099'))
      children = node.generate_child()

      if not node.is_killed():
          print("The children nodes of this node are", end="")

          for child in children:
              if child.state not in explored:
                  print("\nDepth=%d" % child.depth)
                  print(str(child.state))
                  if child.goal_test():
                      print("Which is the goal state\n")
                      graph.add_node(child.graph_node)
                      diff = np.subtract(child.parent.state, child.state)
                      if child.parent.state[2] == 0:
                          diff[0], diff[1] = -diff[0], -diff[1]

                      graph.add_edge(pydot.Edge(child.parent.graph_node, child.graph_node, fontsize='10',
                                                label=str(diff[0]) + "M  " + str(diff[1]) + "C", fontcolor="#cc0099"))

                      # colour all leaves blue
                      leafs = {n.get_name(): True for n in graph.get_nodes()}
                      for e in graph.get_edge_list():
                          leafs[e.get_source()] = False
                      for leaf in leafs:
                          if leafs[leaf] and str(leaf) not in killed and str(leaf) != "\"[0, 0, 0]\"":
                              node = pydot.Node(leaf, style="filled", fillcolor="blue")
                              graph.add_node(node)

                      draw_legend(graph)
                      graph.write_png('MC_BFS.png')

                      return child.find_solution()
                  if child.is_valid():
                      frontier.append(child)
                      explored.append(child.state)

      else:
          print("This node is killed")
          killed.append("\"" + str(node.state) + "\"")

  return

In [ ]:
#Depth First Search Algorithm
import numpy as np
from collections import deque

import pydot

def dfs(initial_state):
    graph = pydot.Dot(graph_type='digraph', label=" Missionaries and Cannibals (DFS) ", color="yellow",
                      fontcolor="black", fontname='Arsenal', style="filled", fillcolor="blue", fontsize="30")
    start_node = Node(initial_state, None, None, 0)
    if start_node.goal_test():
        return start_node.find_solution()

    frontier = deque()
    frontier.append(start_node)
    explored = []
    killed = []

    print("The starting node is \nDepth=%d" % start_node.depth)
    print(str(start_node.state))
    while frontier:
        node = frontier.pop()
        print("\nThe node selected to expand is\nDepth=" + str(node.depth) + "\n" + str(node.state) + "\n")
        explored.append(node.state)
        graph.add_node(node.graph_node)

        if node.parent:
            diff = np.subtract(node.parent.state, node.state)
            if node.parent.state[2] == 0:
                diff[0], diff[1] = -diff[0], -diff[1]
            graph.add_edge(
                pydot.Edge(node.parent.graph_node, node.graph_node, label=str(diff[0]) + "M  " + str(diff[1]) + "C",
                           fontsize='10', fontcolor='#cc0099'))
        children = node.generate_child()
        if not node.is_killed():
            print("The children nodes of this node are", end="")
            for child in children:
                if child.state not in explored:
                    print("\nDepth=%d" % child.depth)
                    print(str(child.state))
                    if child.goal_test():
                        print("which is the goal state\n")
                        graph.add_node(child.graph_node)
                        diff = np.subtract(child.parent.state, child.state)
                        if child.parent.state[2] == 0:
                            diff[0], diff[1] = -diff[0], -diff[1]

                        graph.add_edge(pydot.Edge(child.parent.graph_node, child.graph_node, fontsize='10',
                                                  label=str(diff[0]) + "M  " + str(diff[1]) + "C", fontcolor="#cc0099"))

                        # colour all leaves blue
                        leafs = {n.get_name(): True for n in graph.get_nodes()}
                        for e in graph.get_edge_list():
                            leafs[e.get_source()] = False
                        for leaf in leafs:
                            if leafs[leaf] and str(leaf) not in killed and str(leaf) != "\"[0, 0, 0]\"":
                                node = pydot.Node(leaf, style="filled", fillcolor="blue")
                                graph.add_node(node)

                        draw_legend(graph)
                        graph.write_png('MC_DFS.png')

                        return child.find_solution()
                    if child.is_valid():
                        frontier.append(child)
                        explored.append(child.state)

        else:
            print("This node is killed")
            killed.append("\"" + str(node.state) + "\"")
    return

In [ ]:

from time import time

initial_state = [3, 3, 1]

Node.num_of_instances = 0
t0 = time()
solu = bfs(initial_state)
t1 = time() - t0
print('Solution:', solu)
print('Search Space:', Node.num_of_instances)
print('Time for BFS :', t1, 'seconds')


print("**************************************************************")
print("**************************************************************")

print("DFS Search Space and Solution")

print("**************************************************************")
print("**************************************************************")

t2 = time()
solu = dfs(initial_state)
t3 = time() - t2
print('Solution:', solu)
print('Search Space:', Node.num_of_instances)
print('Time for DFS :', t3, 'seconds')

print("The time taken by DFS is {} and BFS is {} ".format(t3, t1))

The Starting node is 
Depth = 0
[3, 3, 1]

The node selected to expand is
Depth = 0
[3, 3, 1]

The children nodes of this node are
Depth=1
[3, 2, 0]

Depth=1
[3, 1, 0]

Depth=1
[2, 3, 0]

Depth=1
[2, 2, 0]

Depth=1
[1, 3, 0]

The node selected to expand is
Depth = 1
[3, 2, 0]

The children nodes of this node are
Depth=2
[3, 4, 1]

Depth=2
[4, 2, 1]

Depth=2
[4, 3, 1]

Depth=2
[5, 2, 1]

The node selected to expand is
Depth = 1
[3, 1, 0]

The children nodes of this node are
Depth=2
[3, 2, 1]

Depth=2
[4, 1, 1]

Depth=2
[4, 2, 1]

Depth=2
[5, 1, 1]

The node selected to expand is
Depth = 1
[2, 3, 0]

This node is killed

The node selected to expand is
Depth = 1
[2, 2, 0]

The children nodes of this node are
Depth=2
[2, 3, 1]

Depth=2
[2, 4, 1]

Depth=2
[4, 2, 1]

The node selected to expand is
Depth = 1
[1, 3, 0]

This node is killed

The node selected to expand is
Depth = 2
[3, 2, 1]

The children nodes of this node are
Depth=3
[3, 0, 0]

Depth=3
[2, 1, 0]

Depth=3
[1, 2, 0]

The node s